In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

/Users/markhendricks/Projects/finm-portfolio-2023/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,42926953,412799303680,The Trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,9234292,48446230528,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,21697929,21355970560,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,2557,214931952,The fund generally will invest at least 90% of...
QAI,IQ Hedge MultiIQ Hedge Multi-St,ETF,USD,16734,624121728,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,18515962,14783441920,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,506418,2091483648,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,4115950,2840025600,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,4696390,28524961792,The underlying index measures the performance ...


In [5]:
# info = pd.DataFrame(columns=['ETF Description'])
# info.loc['SPY'] = 'Domestic Equity SPDR S&P500'
# info.loc['EFA'] = 'Foreign Equity iShares EAFE'
# info.loc['EEM'] = 'iShares Emerging Markets'
# info.loc['PSP'] = 'Private Equity Invesco Global Private Equity'
# info.loc['QAI'] = 'Absolute Return IQ Multi-Strat'
# info.loc['HYG'] = 'High Yield iShares High Yield Corporate Bond'
# info.loc['DBC'] = 'Invesco DB Commodity Index Tracking Fund'
# info.loc['IYR'] = 'Real Estate iShares US Real Estate'
# info.loc['IEF'] = 'Domestic Bonds iShares 7-10 Year Treasury'
# info.loc['BWX'] = 'Foreign Bonds SPDR Bloomberg Barclay International Treasury'
# info.loc['TIP'] = 'Inflation-Indexed iShares TIPS Bond'
# info.loc['SHV'] = 'iShares Short Treasury Bond'
# info

In [6]:
STARTDATE = '2009-03-31'
ENDDATE = '2023-08-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Adj Close']

[*********************100%%**********************]  12 of 12 completed


In [7]:
prices = df.resample('M').last()

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

In [8]:
rets

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2009-04-30,0.008994,-0.001000,0.155582,0.115190,0.138460,-0.027451,0.296151,0.230203,0.022882,0.000553,0.099346,-0.017952
2009-05-31,0.053672,0.162663,0.159400,0.131918,0.028554,-0.020774,0.022727,0.053892,0.027865,-0.000471,0.058454,0.019966
2009-06-30,0.005149,-0.026259,-0.022495,-0.014049,0.033518,-0.005571,-0.024863,0.042042,-0.003436,0.000599,-0.000655,0.001982
2009-07-31,0.031284,0.018568,0.110146,0.100415,0.069190,0.008317,0.105799,0.143247,0.015326,-0.000027,0.074606,0.000879
2009-08-31,0.007628,-0.040365,-0.013136,0.045031,-0.016968,0.007635,0.131939,0.033413,-0.004151,0.000435,0.036940,0.008414
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,-0.002512,-0.007582,-0.008363,0.029363,0.002021,0.008148,0.009187,0.034619,0.005135,0.003097,0.015975,0.000518
2023-05-31,-0.025960,-0.064092,-0.024023,-0.040071,-0.012314,-0.014383,-0.040266,-0.019120,-0.006471,0.003253,0.004616,-0.011961
2023-06-30,0.008182,0.029479,0.043989,0.044700,0.017802,-0.012553,0.057382,0.046777,0.022626,0.004749,0.064800,-0.003398


In [9]:
retsx

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008441,-0.001553,0.155030,0.114637,0.137907,-0.028004,0.295598,0.229650,0.022329,0.098793,-0.018505
2009-05-31,0.054143,0.163134,0.159871,0.132389,0.029026,-0.020303,0.023198,0.054363,0.028336,0.058925,0.020437
2009-06-30,0.004550,-0.026858,-0.023094,-0.014648,0.032919,-0.006170,-0.025462,0.041443,-0.004035,-0.001254,0.001383
2009-07-31,0.031312,0.018595,0.110173,0.100442,0.069217,0.008344,0.105826,0.143274,0.015353,0.074633,0.000906
2009-08-31,0.007193,-0.040800,-0.013571,0.044595,-0.017404,0.007199,0.131504,0.032977,-0.004586,0.036504,0.007979
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,-0.005609,-0.010679,-0.011460,0.026266,-0.001076,0.005051,0.006090,0.031522,0.002038,0.012878,-0.002579
2023-05-31,-0.029213,-0.067345,-0.027276,-0.043324,-0.015567,-0.017636,-0.043519,-0.022374,-0.009725,0.001363,-0.015214
2023-06-30,0.003433,0.024730,0.039240,0.039952,0.013053,-0.017301,0.052634,0.042028,0.017877,0.060051,-0.008147


In [10]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')